
# TP 1 : Theorie des Graphes 1 
#### L3 INFO NEC 2024–2025 <br> Université de Pau et des Pays de l’Adour
###### date: "2024-12-18"

## prérequis

### installer avec pip (sur la machine)

c'est pareil avec Windows, Linux, Macos

##### version de python :

>si vous avez la version python ou python3 il suffit d'ajoute le 3 en fonction de votre version<br>
>et heuresement python fonctionne avec pip et python3 fonctionne avec pip3 (normalement les versions<br>
>actuelles installées sont pip3 avec python3, python n'est plus utilisé donc pip non plus)<br>

#### installer dash_cytoscape avec 2 packages
```sh
pip3 install dash
pip3 install dash-cytoscape
```

#### installer pip3 :

```sh
python3 -m ensurepip --upgrade
```
enlever 3 dans python3 pour les version ultérieures<br>
si problèmes voir : https://pip.pypa.io/en/stable/installation/



### importation dans le code

In [9]:
# modules importants pour le(s) tp(s) :

# visualisation des couleurs et autres... (partie graphique)
from dash import Dash, html # type: ignore
import dash_cytoscape as cyto # type: ignore

# partie système
from time import time, sleep
import os, sys
from random import randint

# partie mathématique + structure
import numpy as np # type: ignore
from math import inf, sqrt, cos, sin, tan


En premier, il faut définir les variables d'entrée: le graphe. <br>
Ce graphe est donné par un vecteur des sommets, une matrice des arêtes, et un vecteur des poids. <br>

Le graphe avec lequel on va tester notre code est simple: <br>
On va avoir 6 sommets et ça c'est la liste de sommets adjacentes avec les poids : 

### on crée nos classes pour créer une structure de graphe

In [10]:
class Vertex:
    """gère les sommets et ses arrêtes (sous forme d'autres sommets)"""

    id = 0 # identifiant unique de tous les vertex dans les graphes
    
    def __init__(self, name, weight=0):
        """name:le nom du sommet (vertex) associé au noeud
        weight: le poids du sommet pour aller du noeud jusq'au sommet (par défaut=0)
        (nom du noeud non représenté ici)"""
        self.name = name
        self.weight = weight # du sommet du noeud jusq'a ce sommet (ici)
        self.id = Vertex.id
        Vertex.id += 1
    
    def __str__(self):
        '''si on fait print(Vertex) le resultat du print est dans ce return'''
        return self.name

In [11]:
class Node:
    """
    représente les noeuds entre des point dans un graphe
    
    on crée la classe qui s'occupe de tous les noeuds entre 
    les sommets (vertex/vertecies) et leurs autres sommets reliés
    les arrêtes (edges) ne sont pas représentés car c'est la matrice d'adjacence qui s'occupe de ça
    qui est situé dans la classe Graph
    """
    id = 0 # identifiant unique de noeud auquel il appartient    

    def __init__(self, node_vertex:Vertex, vertecies:list[Vertex]):
        self.name = node_vertex.name
        node_vertex.weight = 0 # le poids du sommet du noeud ne peux pas être différent de 0
        self.weight = sum([v.weight for v in vertecies]) # poids du noeud
        self.vertecies = vertecies
        self.degree = len(vertecies)
        self.id = Node.id
        Node.id += 1

    def __str__(self, print_weight:bool=True):
        '''si on affiche Node (print(Node)) renverra ce qui suit'''
        res = f"{self.name} ["
        v_size = len(self.vertecies)
        for i, e in enumerate(self.vertecies):
            res += f"{e.name}"
            if(print_weight):
                res += f" w={e.weight}"
            if(i!=v_size-1):
                if v_size>1:
                    res += "," 
                res+=" "
        res+="]"
        return res
    


In [69]:
class Edge:
    '''représente une arrête entre deux sommets/vertex/noeud'''

    id = 0

    def __init__(self, vfrom:str | int, vto:str | int, weight):
        self.a = vfrom
        self.b = vto
        self.weight = weight
        self.id = Edge.id
        Edge.id += 1
    
    def __eq__(self, edge:object):
        '''compare un edge a un autre objet edge'''
        return self.id == edge.id
    
    def __ne__(self, edge:object):
        return not self.__eq__(edge)

In [217]:

class Graph:

    def __init__(self, data:list[Node], title:str="default"):
        self.title = title # titre du graphique si utilisé
        self.adj = data # liste d'adjacences
        self.edges = []
        self.nodes = [] # liste des noeuds (seulement les noms)
        self.weight = 0 # poids total du graphe    
        self.degree = 0 # pas encore calculé le degré du graphe
        for vx in self.adj:
            self.nodes.append(vx.name)
            self.weight += vx.weight
            for n in vx.vertecies:
                self.edges.append(Edge(vx.name, n.name, n.weight))
                self.degree += 1
        self.create_matrix() # matrice des liens entre les noeuds (Edges) (matrice d'adjacences)

    def create_matrix(self):
        """créer la matrice d'ajdacence des points
        (fonctionne avec les graphes orienté également)
        """
        self.matrix = [[False for _ in self.adj] for _ in self.adj]
        for lin, node in enumerate(self.adj): # parcours noms des noeuds
            for v in node.vertecies: # parcours des "edges"/sommets/vertex
                self.matrix[lin][self.nodes.index(v.name)]=True  

    def __str__(self): # OK
        """si jamais on print un graph (print(Graph)) c'est executé ici
        affichage au plus simple du graphe avec des caractères"""
        res = ""
        for i, node in enumerate(self.adj):
            res += f"{i}\t | {node.name} ["
            v_size = len(node.vertecies)
            for j in range(v_size):
                res += f"{node.vertecies[j].name}"
                if(j!=v_size-1):
                    if v_size>1:
                        res += "," 
                    res+=" "
            res += "]\n"
        return res
    
    def _sort_edges(self, edges:list[Edge])->list[Edge]: # complexite≈O(n+log(5n))
        """fonction privée a ne pas utiliser (en dehors de la classe)
        algorithme reccursif pour les problèmes de pronfondeur et de performances"""
        if len(edges)<2 :
            return edges
        else:
            pivot = edges[len(edges)//2].weight
            l, m, r = [],[],[] # mineurs, égal, majeurs 
            for e in edges:
                if(e.weight < pivot): l.append(e)
                    # si reccursion sur len(m) ce n'est jamais < 2
                    # et donc (boucle infini) dans certains cas                
                elif(e.weight == pivot): m.append(e) 
                else: r.append(e)
            return self._sort_edges(l)+m+self._sort_edges(r)

    def sort_by_weight(self): # OK
        """trie le graphe par poids croissants(asc)
        on trie chaque arrêtes du graphe
        """
        self.edges = self._sort_edges(self.edges)

    def show_edges(self):
        print([e.a+e.b+" w="+str(e.weight) for e in self.edges])
    
    def render(self, layoutname="breadthfirst"):
        """effectue le rendu du graphe visuellement"""
        unique = " "+f'{time()%1e3:.5}'
        app = Dash(self.title+unique)
        custom_layout = {
            'width': '100%', 
            'height': '500px', 
            "border": "3px white solid",
            "border-radius":"5px",
            "background-color":"#666666",
            "title" : {"background-color":"white"}
        }
        styles = [{
                'selector': 'node',  
                'style': {
                    'background-color': '#222222', 
                    'color': 'white',            
                    'label': 'data(label)',       
                    'font-size': '16px',          
                    'text-valign': 'center',     
                    'text-halign': 'center'       
                }
            },
            {
                'selector': 'edge', 
                'style': {
                    'width': 2, 
                    'target-arrow-shape': "vee", 
                    "target-arrow-color": "#4a7cf2",
                    'arrow-scale': 2,
                    'curve-style': "bezier"
                }
            }
        ]
        elems = [] # éléments à afficher (formattés)
        for node in self.adj:
            elems.append({'data': {"id":node.name, "label":node.name}})
        # add edges
        for edge in self.edges:
            elems.append({'data': {'source': edge.a, 'target': edge.b}})
        app.layout = html.Div([
            cyto.Cytoscape(
                id='cytoscape'+unique,
                elements=elems,
                layout={'name': layoutname},
                style=custom_layout,
                stylesheet=styles
            )
        ])
        print("également ouvert sur la page \"localhost:8050\"")        
        app.run_server(debug=True)


In [218]:
# Tests :

# tester l'efficacité des calculs avec les maths

# fonction temporaire pour générer une liste de sommets aléatoire (avec edges aléatoire)
# sans avoir de boucle sur le même noeud (paramètre v) exemple: 'v'->'v'
tmpadata2 = lambda v : [Vertex(chr(i+65), randint(1,400)) for i in range(randint(1,26)) if i != v]
data2 = [Node(Vertex(chr(j+65)), tmpadata2(j)) for j in range(26)]
graphe2 = Graph(data2)

print(graphe2)
graphe2.show_edges()
graphe2.sort_by_weight()
graphe2.show_edges()



0	 | A [B, C, D, E]
1	 | B [A, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V]
2	 | C [A, B]
3	 | D [A, B, C, E, F, G, H, I, J, K, L, M, N, O, P, Q, R]
4	 | E [A, B, C]
5	 | F [A, B, C, D, E, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W]
6	 | G [A, B, C, D, E, F, H]
7	 | H [A, B]
8	 | I [A, B, C, D, E, F, G, H, J, K]
9	 | J [A, B, C, D, E, F]
10	 | K [A]
11	 | L [A, B, C]
12	 | M [A, B, C, D, E, F, G, H, I, J, K, L, N, O, P, Q, R, S, T, U, V, W, X, Y]
13	 | N [A, B, C, D, E, F, G, H, I, J, K, L, M, O, P, Q, R, S, T, U, V, W, X, Y]
14	 | O [A, B, C, D, E, F, G, H, I, J, K]
15	 | P [A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, Q, R, S, T, U, V, W]
16	 | Q [A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, R, S, T, U]
17	 | R [A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, S, T, U, V, W]
18	 | S [A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, T, U, V, W, X]
19	 | T [A, B, C, D, E, F, G, H]
20	 | U [A, B, C, D, E, F, G, H, I, J, K, L, M, N, O]
21	 | V [A, B, C, D]

In [219]:
# breadthfirst(default) grid circle concentric cose random preset
graphe2.render("grid")

également ouvert sur la page "localhost:8050"


## saisie des données

#### données fournies :

> 1: 2(2), 3(1)<br>
> 2: 1(2), 4(2), 5(3)<br>
> 3: 1(1), 2(3), 4(2)<br>
> 4: 2(2), 3(5), 5(2), 6(4)<br>
> 5: 2(3), 4(2), 6(2)<br>
> 6: 4(2), 5(2)<br>


In [73]:
# rappel format du noeud : 
# Noeud(nomActuel, [Vertex("nomLié1", poid1),Vertex("nomLié2", poid2),etc...])
data = [
    Node(Vertex("1"),[Vertex("2",2),Vertex("3",1)]),
    Node(Vertex("2"),[Vertex("1",2),Vertex("4",2),Vertex("5",3)]),
    Node(Vertex("3"),[Vertex("1",1),Vertex("2",3),Vertex("4",2)]),
    Node(Vertex("4"),[Vertex("2",2),Vertex("3",5),Vertex("5",2),Vertex("6",4)]),
    Node(Vertex("5"),[Vertex("2",3),Vertex("4",2),Vertex("6",2)]),
    Node(Vertex("6"),[Vertex("4",2),Vertex("5",2)])
]
graphe = Graph(data,"exemple de graphe")
print(np.matrix(graphe.matrix))


# on vérifie que tout est juste
ok = [[False, True, True, False, False, False],
      [True, False, False, True, True, False],
      [True, True, False, True, False, False],
      [False, True, True, False, True, True],
      [False, True, False, True, False, True],
      [False, False, False, True, True, False]]
assert ok == graphe.matrix, "une des valeurs n'est pas vraie"

[[False  True  True False False False]
 [ True False False  True  True False]
 [ True  True False  True False False]
 [False  True  True False  True  True]
 [False  True False  True False  True]
 [False False False  True  True False]]


## Kruskal

> init: arrêtes d'ordre ascendant de poids

on peut utiliser `sort()` qui existe a la fois dans python et dans RStudio<br>
> pour i=1...n-1 des sommets<br>
> &emsp;prendre l'arrête de poids min qui ne fait pas une boucle<br>
> &emsp;et qui n'est pas déja dans la liste des arrêtes que l'on a déja choisit<br>
> fin

In [74]:
# on fait de l'héritage car on peut pas 
# rajouter la méthode kruskal comme en swift avec des extensions
class GraphK(Graph):

    # obligatoire (heritage)
    def __init__(self, data, title = "default"):
        super().__init__(data, title)

    # obligatoire
    def __init__(self, gr:Graph):
        """convertit un Graph normal en GraphK"""
        super().__init__(gr.adj,gr.title)
    
    def kruskal(self,red_tarjan_rule=False)->list[any]:
        """renvoie l'arbre couvrant de poids minimal
        applique la règle rouge de tarjan si booléen est True (False par défaut)"""
        # comme vue en cours
        self.sort_by_weight() # trie tout le graphe
        tmin = []
        carry = True # stopper la boucle
        idx = 0
        for node in self.adj: # pour chaque sommets (du noeud)
            # 1er arrete qui n'est pas dans tmin (forcément la min)
            while(carry and idx < len(tmin)): # parcours de tmin
                if(tmin[idx] != ):
                    
                idx+=1
            # reset
            carry=True
            idx = 0

            a = node.vertecies[0] 


        

In [18]:
# vérification (données insérées a la main pour être sûr)
graphe = GraphK(graphe) # conversion
print(graphe)
#graphe.sort_weight_ASC()
#print(graphe)

NameError: name 'graphe' is not defined

methode kruskall en R : 
```R
kruskal <- function(sommets,arretes,poids){
    poids_ord <- sort(poids,index.return=true)
    poids <- poids_ord$x
    index_poids <- poids_ord$ix 
    return poids
}
```

In [117]:
from dash import Dash, html
import dash_cytoscape as cyto

app = Dash()

directed_edges = [
    {'data': {'id': src+tgt, 'source': src, 'target': tgt}}
    for src, tgt in ['BA', 'BC', 'CD', 'DA']
]

directed_elements = [{'data': {'id': id_}} for id_ in 'ABCD'] + directed_edges

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-styling-9',
        layout={'name': 'circle'},
        style={'width': '100%', 'height': '400px'},
        elements=directed_elements,
        stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'label': 'data(id)'
                }
            },
            {
                'selector': 'edge',
                'style': {
                    # The default curve style does not work with certain arrows
                    'curve-style': 'bezier'
                }
            },
            {
                'selector': '#',
                'style': {
                    'source-arrow-color': 'red',
                    'source-arrow-shape': 'triangle',
                    'line-color': 'red'
                }
            },
            {
                'selector': '#DA',
                'style': {
                    'target-arrow-color': 'blue',
                    'target-arrow-shape': 'vee',
                    'line-color': 'blue'
                }
            },
            {
                'selector': '#BC',
                'style': {
                    'mid-source-arrow-color': 'green',
                    'mid-source-arrow-shape': 'diamond',
                    'mid-source-arrow-fill': 'hollow',
                    'line-color': 'green',
                }
            },
            {
                'selector': '#CD',
                'style': {
                    'mid-target-arrow-color': 'black',
                    'mid-target-arrow-shape': 'circle',
                    'arrow-scale': 2,
                    'line-color': 'black',
                }
            }
        ]
    )
])

app.run(debug=True)
